In [1]:
import os
from  dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)


True

In [16]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma

In [19]:
file = 'files/products_min.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [23]:
query ="List some products from the document"
response = index.query(query)

In [24]:
response

' Homesake Gold-Toned & White Solid Handcrafted Table Lamp with Shade, Being Human Clothing Navy Printed Casual Slim Shirt, ID Men Beige Leather Derbys, ID Men Brown Leather Formal Slip-Ons'

In [20]:
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma
).from_loaders([loader])

In [26]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [27]:
docs = loader.load()

In [28]:
docs[0]

Document(page_content=': 0\nProductID: 10017413\nProductName: DKNY Unisex Black & Grey Printed Medium Trolley Bag\nProductBrand: DKNY\nGender: Unisex\nPrice (INR): 11745\nNumImages: 7\nDescription: Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer\nPrimaryColor: Black', metadata={'source': 'files/products_min.csv', 'row': 0})

In [29]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [30]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [31]:
print(len(embed))

1536


In [32]:
print(embed[:5])

[-0.021898041792938288, 0.006773174083663193, -0.018200152971572344, -0.03911712282827725, -0.014062040088796447]


In [33]:
db = Chroma.from_documents(
    docs, 
    embeddings
)

In [34]:
query = "Please suggest a shirt with sunblocking"
docs = db.similarity_search(query)

In [35]:
docs[0]

Document(page_content=': 15\nProductID: 1000795\nProductName: Being Human Clothing Navy Printed Casual Slim Shirt\nProductBrand: Being Human\nGender: Men\nPrice (INR): 1079\nNumImages: 6\nDescription: Navy blue printedÂ\xa0casual shirt, has a spread collar, a full button placket, two pockets,Â\xa0long sleeves, a curved hem\nPrimaryColor: Navy', metadata={'row': 15, 'source': 'files/products_min.csv'})

In [36]:
retriever = db.as_retriever()

In [40]:
llm = ChatOpenAI(temperature = 0.0, model='gpt-3.5-turbo')


In [37]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [43]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts in a table in markdown and summarize each one.") 


In [45]:
print(response)

| ProductID | ProductName | ProductBrand | Gender | Price (INR) | NumImages | Description | PrimaryColor |
|-----------|-------------|--------------|--------|-------------|-----------|-------------|--------------|
| 1000795   | Being Human Clothing Navy Printed Casual Slim Shirt | Being Human | Men | 1079 | 6 | Navy blue printed casual shirt, has a spread collar, a full button placket, two pockets, long sleeves, a curved hem | Navy |
| 10015839  | Raymond Men Blue Slim Fit Printed Formal Shirt | Raymond | Men | 1079 | 5 | Blue printed formal shirt, has a spread collar, long sleeves, button placket, straight hem, one patch pocket | Blue |

Summary:
1. Being Human Clothing Navy Printed Casual Slim Shirt: This navy blue printed casual shirt for men features a spread collar, full button placket, two pockets, long sleeves, and a curved hem. It is from the brand Being Human and is priced at INR 1079. It comes with 6 images and the primary color is navy.
2. Raymond Men Blue Slim Fit Printed F

In [46]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [47]:
query =  "Please list all your shirts in a table in markdown and summarize each one."

In [48]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [49]:
response

'| ProductID | ProductName | ProductBrand | Gender | Price (INR) | NumImages | Description | PrimaryColor |\n|-----------|-------------|--------------|--------|-------------|-----------|-------------|--------------|\n| 1000795   | Being Human Clothing Navy Printed Casual Slim Shirt | Being Human | Men | 1079 | 6 | Navy blue printed casual shirt, has a spread collar, a full button placket, two pockets, long sleeves, a curved hem | Navy |\n| 10000245  | Parx Men Green Printed Polo Collar T-shirt | Parx | Men | 629 | 5 | Green printed T-shirt, has a polo collar, and short sleeves | Green |\n\nSummary:\n1. Being Human Clothing Navy Printed Casual Slim Shirt: This navy blue printed casual shirt for men features a spread collar, full button placket, two pockets, long sleeves, and a curved hem. It is priced at INR 1079.\n2. Parx Men Green Printed Polo Collar T-shirt: This green printed T-shirt for men has a polo collar and short sleeves. It is priced at INR 629.'